In [1]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)


# import data

In [2]:
data = pd.read_parquet("outputs/dedupe_data_out.parquet")
data

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,None,parent,None,None,None,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,None,parent,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,None,parent,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,None,parent,None,None,None,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,None,parent,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8410089,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,None,parent,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None
8410090,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,None,parent,None,dt_froedge,particle mass ratios,froedge|d,None
8410091,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,None,parent,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None
8410092,openalex::W999790414,viXra,openalex,None,None,https://vixra.org/pdf/1306.0105v3.pdf,Investigation of the Formalism of Particle Dyn...,Chi-Yi Chen,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2013-06-01,2013,None,parent,None,chen_chi_yi,investigation of the formalism of particle dyn...,chen|c,None


In [55]:
# any cluster id should correspond to one connected component
print(data["records_hierarchy_copy"].value_counts())


records_hierarchy_copy
parent                                          7931373
review                                           175443
part_of                                          149454
version                                          105977
publish_version                                    9351
mirror (arXiv)                                     8286
mirror (AgEcon Search)                             6702
mirror (Open Science Framework)                    4356
mirror (PsyArXiv)                                  4304
mirror (INA-Rxiv)                                  3768
mirror (SocArXiv)                                  3395
child                                              2028
mirror (ResearchGate)                              1633
mirror (Law Archive)                                935
mirror (EarthArXiv)                                 410
correction                                          355
mirror (Thesis Commons)                             346
comment                  

In [56]:
['others', 'part_of', 'comment', 'review']

['others', 'part_of', 'comment', 'review']

In [53]:
bad_values = {"others", "part_of", "comment", "review"}

mask = ~data["records_hierarchy_copy"].astype(str).str.contains(
    r"\b(" + "|".join(bad_values) + r")\b",
    case=False,
    regex=True
)

data_clean = data[mask].copy()


/tmp/ipykernel_23490/4120613175.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = ~data["records_hierarchy_copy"].astype(str).str.contains(


In [57]:
len(data_clean)

8084860

In [58]:
clustered = data_clean[data_clean["dup_group_id"].notna()].copy()

n_clusters = clustered["dup_group_id"].nunique()
sizes = clustered.groupby("dup_group_id")["record_id"].size()

print("clusters:", n_clusters)
print("min/median/p95/max:", sizes.min(), sizes.median(), sizes.quantile(0.95), sizes.max())


clusters: 7464505
min/median/p95/max: 1 1.0 2.0 702


In [59]:
# any cluster id should correspond to one connected component
print(data_clean["dup_group_id"].nunique(dropna=True))


7464505


In [60]:
sizes = data_clean.dropna(subset=["dup_group_id"]).groupby("dup_group_id")["record_id"].size()
print(sizes.describe())


count    7.464505e+06
mean     1.083107e+00
std      6.625238e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      7.020000e+02
Name: record_id, dtype: float64


In [61]:
top = sizes.sort_values(ascending=False).head(20)
print(top)


dup_group_id
datacite::10.5281/zenodo.14289192             702
datacite::10.5281/zenodo.15609432             570
datacite::10.22004/ag.econ.138684             450
datacite::10.22004/ag.econ.136112             390
datacite::10.22004/ag.econ.286599             301
datacite::10.5281/zenodo.15832876             286
datacite::10.5281/zenodo.15161515             261
crossref::10.31219/osf.io/29d34               210
crossref::10.32388/045113                     184
datacite::10.5281/zenodo.10011722             182
datacite::10.5281/zenodo.14599183             177
datacite::10.5281/zenodo.15631517             167
crossref::10.2139/ssrn.2721313                164
datacite::10.5281/zenodo.15690627             159
crossref::10.31237/osf.io/7j3am               150
crossref::10.21055/preprints-3112015          145
crossref::10.20944/preprints201908.0037.v2    141
crossref::10.21055/preprints-3112061          136
crossref::10.31237/osf.io/354ab               128
datacite::10.5281/zenodo.17172763    

# unify/merge group ids across stages

In [62]:
import pandas as pd

class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, x):
        x = str(x)
        if x not in self.parent:
            self.parent[x] = x
        # path compression
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            # deterministic: attach larger root to smaller root (stable)
            if ra < rb:
                self.parent[rb] = ra
            else:
                self.parent[ra] = rb

def merge_groups_from_parent_links(
    df: pd.DataFrame,
    *,
    record_id_col="record_id",
    parent_id_col="parent_record_id",
    hierarchy_col="records_hierarchy",
    group_id_col="dup_group_id",
    only_when_duplicate_prefix="parent - duplicate",
) -> pd.DataFrame:
    """
    Merge cluster IDs so that any duplicate link (child -> parent)
    forces both records to share a single canonical dup_group_id.

    Works even if different stages created different group ids.
    """
    # rows that are duplicates (children)
    h = df[hierarchy_col].astype("string").fillna("")
    dup_mask = h.str.startswith(only_when_duplicate_prefix)

    # edges child -> parent
    edges = df.loc[dup_mask, [record_id_col, parent_id_col]].dropna()
    if edges.empty:
        return df

    uf = UnionFind()

    # Union by record IDs (most robust)
    for child, parent in edges.itertuples(index=False):
        uf.union(child, parent)

    # Map each record_id to component root
    roots = df[record_id_col].astype("string").map(uf.find)

    # Canonical group id = component root (or hash of it if you prefer)
    # This ensures *all stages collapse into one group id*.
    df[group_id_col] = roots

    # Optional: ensure parents have parent_id = NA (clean)
    parent_mask = h.str.lower().str.strip().eq("parent")
    if parent_id_col in df.columns:
        df.loc[parent_mask, parent_id_col] = pd.NA

    return df


In [63]:
# after stage i
data_group_merge = merge_groups_from_parent_links(data_clean)


In [64]:
clustered = data_group_merge[data_group_merge["dup_group_id"].notna()].copy()

n_clusters = clustered["dup_group_id"].nunique()
sizes = clustered.groupby("dup_group_id")["record_id"].size()

print("clusters:", n_clusters)
print("min/median/p95/max:", sizes.min(), sizes.median(), sizes.quantile(0.95), sizes.max())


clusters: 7464507
min/median/p95/max: 1 1.0 2.0 702


In [65]:
sizes = data_group_merge.dropna(subset=["dup_group_id"]).groupby("dup_group_id")["record_id"].size()
print(sizes.describe())

count    7.464507e+06
mean     1.083107e+00
std      6.625225e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      7.020000e+02
Name: record_id, dtype: float64


In [66]:
top = sizes.sort_values(ascending=False).head(20)
print(top)


dup_group_id
datacite::10.5281/zenodo.14289192             702
datacite::10.5281/zenodo.15609432             570
datacite::10.22004/ag.econ.138684             450
datacite::10.22004/ag.econ.136112             390
datacite::10.22004/ag.econ.286599             301
datacite::10.5281/zenodo.15832876             286
datacite::10.5281/zenodo.15161515             261
crossref::10.31219/osf.io/29d34               210
crossref::10.32388/045113                     184
datacite::10.5281/zenodo.10011722             182
datacite::10.5281/zenodo.14599183             177
datacite::10.5281/zenodo.15631517             167
crossref::10.2139/ssrn.2721313                164
datacite::10.5281/zenodo.15690627             159
crossref::10.31237/osf.io/7j3am               150
crossref::10.21055/preprints-3112015          145
crossref::10.20944/preprints201908.0037.v2    141
crossref::10.21055/preprints-3112061          136
datacite::10.5281/zenodo.17172763             128
crossref::10.31237/osf.io/354ab      

# creates inspection packets + an HTML report (clickable) + the key CSVs, all in outputs/

In [67]:
from __future__ import annotations

from pathlib import Path
import json
import random
import pandas as pd
import re
import hashlib
def _gid_slug(gid: str) -> str:
    safe = _safe_filename(gid, 60)
    h = hashlib.md5(str(gid).encode("utf-8")).hexdigest()[:10]
    return f"{safe}__{h}"

_BAD_FS_CHARS = re.compile(r'[<>:"/\\|?*\x00-\x1F]')  # Windows-illegal + control chars

def _safe_filename(s: str, max_len: int = 160) -> str:
    """
    Make a string safe to use as a filename on Windows/macOS/Linux.
    - Replaces illegal characters (including / and :) with '_'
    - Collapses whitespace
    - Truncates to avoid path-too-long issues
    """
    s = str(s)
    s = _BAD_FS_CHARS.sub("_", s)
    s = re.sub(r"\s+", " ", s).strip()
    if len(s) > max_len:
        s = s[:max_len].rstrip("_ ").strip()
    return s or "empty"


def export_inspection_packets(
    df: pd.DataFrame,
    out_dir: str | Path = "outputs",
    *,
    # ---- columns (edit if yours differ)
    record_id_col: str = "record_id",
    group_id_col: str = "dup_group_id",
    hierarchy_col: str = "records_hierarchy",
    parent_id_col: str = "parent_record_id",
    title_col: str = "title",
    authors_col: str = "authors_flat",
    server_col: str = "server_name",
    date_col: str | None = "date_first_seen",   # set None if you don't have it
    title_clean_col: str | None = None,         # e.g. "title_clean_v2" if you saved it
    # ---- packets
    n_random: int = 10,
    n_largest: int = 10,
    n_cross_server: int = 10,
    big_cluster_threshold: int = 50,            # red flag threshold
    random_seed: int = 13,
) -> dict:
    """
    Export dedupe clusters for manual inspection and an audit report.

    Outputs:
      outputs/
        dedupe_clusters_long.csv
        dedupe_clusters_summary.csv
        dedupe_audit_report.json
        dedupe_report.html
        packets/
          10_random_clusters/*.csv
          10_largest_clusters/*.csv
          10_cross_server_clusters/*.csv
          red_flags/*.csv
    Returns:
      report dict (also saved to JSON).
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    packets_dir = out_dir / "packets"
    packets_dir.mkdir(exist_ok=True)

    # -----------------------------
    # helpers
    # -----------------------------
    def _safe_cols(cols: list[str]) -> list[str]:
        return [c for c in cols if c in df.columns]

    def _count_duplicates(s: pd.Series) -> int:
        ss = s.astype("string").fillna("")
        return int(ss.str.startswith("parent - duplicate").sum())

    def _is_nonempty_string_series(s: pd.Series) -> pd.Series:
        ss = s.astype("string")
        return ss.notna() & (ss.str.len() > 0)

    # -----------------------------
    # clusterable rows
    # -----------------------------
    if group_id_col not in df.columns:
        raise ValueError(f"Missing required column: {group_id_col}")

    clustered_mask = _is_nonempty_string_series(df[group_id_col])
    clustered = df.loc[clustered_mask].copy()

    # -----------------------------
    # clusters_long (one row per record)
    # -----------------------------
    base_cols = [
        group_id_col,
        record_id_col,
        parent_id_col,
        hierarchy_col,
        server_col,
        date_col if date_col else "",
        title_col,
        title_clean_col if title_clean_col else "",
        authors_col,
    ]
    cols = _safe_cols([c for c in base_cols if c])

    clusters_long = clustered.loc[:, cols].copy()

    # cluster size per row
    clusters_long["cluster_size"] = clusters_long.groupby(group_id_col)[record_id_col].transform("size")

    # sort
    sort_cols = [group_id_col, "cluster_size"]
    asc = [True, False]
    if date_col and date_col in clusters_long.columns:
        clusters_long[date_col] = pd.to_datetime(clusters_long[date_col], errors="coerce")
        sort_cols.append(date_col)
        asc.append(True)

    clusters_long = clusters_long.sort_values(sort_cols, ascending=asc)
    clusters_long_path = out_dir / "dedupe_clusters_long.csv"
    clusters_long.to_csv(clusters_long_path, index=False)    
    clusters_long.to_pickle(out_dir / "dedupe_clusters_long.pkl")

    # -----------------------------
    # clusters_summary (one row per cluster)
    # -----------------------------
    def _join_uniq(x: pd.Series) -> str:
        vals = [str(v) for v in x.dropna().tolist()]
        return ";".join(sorted(set(vals)))

    agg = {record_id_col: "count"}
    if server_col in clusters_long.columns:
        agg[server_col] = _join_uniq
    if title_col in clusters_long.columns:
        agg[title_col] = "first"
    if date_col and date_col in clusters_long.columns:
        agg[date_col] = "min"

    cluster_summary = (
        clusters_long.groupby(group_id_col, dropna=False)
        .agg(agg)
        .rename(columns={
            record_id_col: "n_records",
            server_col: "servers",
            title_col: "example_title",
            date_col: "min_date",
        })
        .reset_index()
        .sort_values("n_records", ascending=False)
    )

    summary_path = out_dir / "dedupe_clusters_summary.csv"
    cluster_summary.to_csv(summary_path, index=False)

    # -----------------------------
    # audit report (JSON)
    # -----------------------------
    report = {}
    report["n_rows_total"] = int(len(df))
    report["n_rows_clustered"] = int(len(clustered))

    if hierarchy_col in df.columns:
        h = df[hierarchy_col].astype("string")
        report["n_parents"] = int(h.str.lower().eq("parent").sum())
        report["n_duplicates"] = _count_duplicates(df[hierarchy_col])
        report["n_unlabeled"] = int(df[hierarchy_col].isna().sum())
    else:
        report["n_parents"] = report["n_duplicates"] = report["n_unlabeled"] = 0

    report["n_clusters"] = int(cluster_summary[group_id_col].nunique()) if len(cluster_summary) else 0

    if len(cluster_summary):
        sizes = cluster_summary["n_records"]
        report["cluster_size_min"] = int(sizes.min())
        report["cluster_size_median"] = float(sizes.median())
        report["cluster_size_p95"] = float(sizes.quantile(0.95))
        report["cluster_size_max"] = int(sizes.max())
    else:
        report["cluster_size_min"] = report["cluster_size_median"] = report["cluster_size_p95"] = report["cluster_size_max"] = 0

    # cross-server clusters
    if "servers" in cluster_summary.columns:
        # servers field is "a;b;c" - count distinct servers
        n_srv = cluster_summary["servers"].fillna("").apply(lambda x: len([z for z in str(x).split(";") if z]))
        report["clusters_cross_server"] = int((n_srv >= 2).sum())
        report["clusters_single_server"] = int((n_srv == 1).sum())
    else:
        report["clusters_cross_server"] = report["clusters_single_server"] = 0

    # parent integrity checks
    if parent_id_col in df.columns and hierarchy_col in df.columns:
        dup_mask = df[hierarchy_col].astype("string").fillna("").str.startswith("parent - duplicate")
        report["dups_missing_parent_id"] = int(df.loc[dup_mask, parent_id_col].isna().sum())

        parent_mask = df[hierarchy_col].astype("string").fillna("").str.lower().eq("parent")
        report["parents_with_parent_id_filled"] = int(df.loc[parent_mask, parent_id_col].notna().sum())
    else:
        report["dups_missing_parent_id"] = 0
        report["parents_with_parent_id_filled"] = 0

    audit_json_path = out_dir / "dedupe_audit_report.json"
    with open(audit_json_path, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)

    # -----------------------------
    # red flags exports
    # -----------------------------
    red_flags_dir = packets_dir / "red_flags"
    red_flags_dir.mkdir(exist_ok=True)

    # 1) duplicates missing parent id
    if parent_id_col in df.columns and hierarchy_col in df.columns:
        dup_mask = df[hierarchy_col].astype("string").fillna("").str.startswith("parent - duplicate")
        bad = df.loc[dup_mask & df[parent_id_col].isna(), _safe_cols([record_id_col, group_id_col, hierarchy_col, server_col, title_col])].copy()
        bad.to_csv(red_flags_dir / "dups_missing_parent_id.csv", index=False)

    # 2) big clusters
    if len(cluster_summary):
        big_ids = cluster_summary.loc[cluster_summary["n_records"] >= big_cluster_threshold, group_id_col].tolist()
        if big_ids:
            big = clusters_long.loc[clusters_long[group_id_col].isin(big_ids), _safe_cols([group_id_col, record_id_col, parent_id_col, hierarchy_col, server_col, title_col])].copy()
            big.to_csv(red_flags_dir / f"big_clusters_{big_cluster_threshold}plus.csv", index=False)

    # -----------------------------
    # packet exporters
    # -----------------------------
    # def _export_cluster_csvs(cluster_ids: list[str], folder: Path):
    #     folder.mkdir(exist_ok=True)
    #     for gid in cluster_ids:
    #         part = clusters_long[clusters_long[group_id_col] == gid]
    #         part.to_csv(folder / f"cluster_{gid}.csv", index=False)

    def _export_cluster_csvs(cluster_ids, folder):
        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)  # <--- IMPORTANT
    
        for gid in cluster_ids:
            part = clusters_long[clusters_long[group_id_col].astype(str) == str(gid)]
            safe_gid = _safe_filename(gid)
            part.to_csv(folder / f"cluster_{safe_gid}.csv", index=False)
            # part.to_csv(folder / f"cluster_{_gid_slug(safe_gid)}.csv", index=False)

    
    # random clusters (from all clusters)
    random.seed(random_seed)
    all_ids = cluster_summary[group_id_col].astype(str).tolist()
    random_ids = random.sample(all_ids, k=min(n_random, len(all_ids))) if all_ids else []
    _export_cluster_csvs(random_ids, packets_dir / f"{len(random_ids)}_random_clusters")

    # largest clusters
    largest_ids = cluster_summary.head(n_largest)[group_id_col].astype(str).tolist()
    _export_cluster_csvs(largest_ids, packets_dir / f"{len(largest_ids)}_largest_clusters")

    # cross-server clusters
    cross_ids = []
    if "servers" in cluster_summary.columns:
        n_srv = cluster_summary["servers"].fillna("").apply(lambda x: len([z for z in str(x).split(";") if z]))
        cross_df = cluster_summary.loc[n_srv >= 2].copy()
        cross_ids = cross_df.head(n_cross_server)[group_id_col].astype(str).tolist()
    _export_cluster_csvs(cross_ids, packets_dir / f"{len(cross_ids)}_cross_server_clusters")

    # -----------------------------
    # simple HTML report
    # -----------------------------
    html_path = out_dir / "dedupe_report.html"

    def _escape(x) -> str:
        return ("" if pd.isna(x) else str(x)).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

    top_preview = cluster_summary.head(50).copy()
    # make HTML table
    rows = []
    for _, r in top_preview.iterrows():
        gid = _escape(r[group_id_col])
        nrec = _escape(r.get("n_records", ""))
        srv = _escape(r.get("servers", ""))
        title = _escape(r.get("example_title", ""))
        rows.append(f"<tr><td>{gid}</td><td>{nrec}</td><td>{srv}</td><td>{title}</td></tr>")

    html = f"""
<!doctype html>
<html>
<head>
<meta charset="utf-8"/>
<title>Dedupe Audit Report</title>
<style>
body {{ font-family: Arial, sans-serif; margin: 20px; }}
code {{ background: #f4f4f4; padding: 2px 4px; border-radius: 4px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f7f7f7; }}
.small {{ color: #555; font-size: 12px; }}
</style>
</head>
<body>
<h2>Dedupe Audit Report</h2>

<h3>Summary</h3>
<ul>
  <li>Total rows: <b>{report['n_rows_total']}</b></li>
  <li>Clustered rows: <b>{report['n_rows_clustered']}</b></li>
  <li>Clusters: <b>{report['n_clusters']}</b></li>
  <li>Parents: <b>{report['n_parents']}</b> | Duplicates: <b>{report['n_duplicates']}</b> | Unlabeled: <b>{report['n_unlabeled']}</b></li>
  <li>Cluster size (min/median/p95/max): <b>{report['cluster_size_min']}</b> / <b>{report['cluster_size_median']:.1f}</b> / <b>{report['cluster_size_p95']:.1f}</b> / <b>{report['cluster_size_max']}</b></li>
  <li>Cross-server clusters: <b>{report['clusters_cross_server']}</b> | Single-server clusters: <b>{report['clusters_single_server']}</b></li>
  <li>Duplicates missing parent_id: <b>{report['dups_missing_parent_id']}</b></li>
</ul>

<p class="small">
Files saved in <code>{out_dir.as_posix()}</code>:
<br/>- dedupe_clusters_long.csv
<br/>- dedupe_clusters_summary.csv
<br/>- dedupe_audit_report.json
<br/>- packets/...
</p>

<h3>Top 50 largest clusters (preview)</h3>
<table>
  <thead><tr><th>dup_group_id</th><th>n_records</th><th>servers</th><th>example_title</th></tr></thead>
  <tbody>
    {''.join(rows)}
  </tbody>
</table>

</body>
</html>
""".strip()

    html_path.write_text(html, encoding="utf-8")

    # return
    report["files_written"] = {
        "clusters_long": str(clusters_long_path),
        "clusters_summary": str(summary_path),
        "audit_json": str(audit_json_path),
        "html_report": str(html_path),
        "packets_dir": str(packets_dir),
    }
    return report


In [68]:
report = export_inspection_packets(
    data_group_merge,
    out_dir="outputs",
    date_col="date_first_seen",   # or None
    title_clean_col="title_clean_v2",  # optional
    n_random=10,
    n_largest=10,
    n_cross_server=10,
    big_cluster_threshold=50,
)
report


{'n_rows_total': 8084860,
 'n_rows_clustered': 8084860,
 'n_parents': 7436474,
 'n_duplicates': 620463,
 'n_unlabeled': 0,
 'n_clusters': 7464507,
 'cluster_size_min': 1,
 'cluster_size_median': 1.0,
 'cluster_size_p95': 2.0,
 'cluster_size_max': 702,
 'clusters_cross_server': 193283,
 'clusters_single_server': 7271224,
 'dups_missing_parent_id': 0,
 'parents_with_parent_id_filled': 0,
 'files_written': {'clusters_long': 'outputs/dedupe_clusters_long.csv',
  'clusters_summary': 'outputs/dedupe_clusters_summary.csv',
  'audit_json': 'outputs/dedupe_audit_report.json',
  'html_report': 'outputs/dedupe_report.html',
  'packets_dir': 'outputs/packets'}}

In [69]:
from pathlib import Path
OUT_DIR = Path("outputs"); OUT_DIR.mkdir(exist_ok=True)

cols_new = [
    "record_id",
    'server_name',
    "records_hierarchy_copy2",
    "records_hierarchy_copy",
    "records_hierarchy",
    "parent_record_id",
    "dup_group_id",
    "authors_fp_tokenbag",
    "authors_fp_last_initial",
    "authors_fp_last",
    "title_clean_v2",
]
cols_new = [c for c in cols_new if c in data_group_merge.columns]

data_group_merge[cols_new].to_parquet(OUT_DIR / "dedupe_new_columns.parquet", index=False)
print("Saved:", OUT_DIR / "dedupe_new_columns.parquet", "| cols:", cols_new)


Saved: outputs/dedupe_new_columns.parquet | cols: ['record_id', 'server_name', 'records_hierarchy_copy', 'records_hierarchy', 'parent_record_id', 'dup_group_id', 'authors_fp_tokenbag', 'authors_fp_last_initial', 'authors_fp_last', 'title_clean_v2']


In [70]:
data_group_merge["records_hierarchy"].value_counts(dropna=False)#.head(60)

records_hierarchy
parent                                               7436474
parent - duplicate (SSRN)                             176423
parent - duplicate (Zenodo)                            94299
parent - duplicate (arXiv)                             61901
parent - duplicate (Research Square)                   43381
                                                      ...   
mirror (BodoArXiv)                                         1
mirror (TechRxiv)                                          1
parent - duplicate (Prepublicaciones OpenCiencia)          1
parent - duplicate (Therapoid)                             1
parent - duplicate (searchRxiv)                            1
Name: count, Length: 145, dtype: int64

In [71]:
data_clean["records_hierarchy"].value_counts(dropna=False)#.head(60)

records_hierarchy
parent                                               7436474
parent - duplicate (SSRN)                             176423
parent - duplicate (Zenodo)                            94299
parent - duplicate (arXiv)                             61901
parent - duplicate (Research Square)                   43381
                                                      ...   
mirror (BodoArXiv)                                         1
mirror (TechRxiv)                                          1
parent - duplicate (Prepublicaciones OpenCiencia)          1
parent - duplicate (Therapoid)                             1
parent - duplicate (searchRxiv)                            1
Name: count, Length: 145, dtype: int64

In [23]:
data_group_merge

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,<NA>,parent,crossref::10.21467/preprints.48,None,None,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,<NA>,parent,crossref::10.21467/preprints.43,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,<NA>,parent,crossref::10.21467/preprints.39,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,<NA>,parent,crossref::10.21467/preprints.38,None,None,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,<NA>,parent,crossref::10.21467/preprints.36,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8410089,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,<NA>,parent,openalex::W999325625,lubomir_vlcek,three objections to modern physics,vlcek|l,None
8410090,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,<NA>,parent,openalex::W999460032,dt_froedge,particle mass ratios,froedge|d,None
8410091,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,<NA>,parent,openalex::W99967155,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None
8410092,openalex::W999790414,viXra,openalex,None,None,https://vixra.org/pdf/1306.0105v3.pdf,Investigation of the Formalism of Particle Dyn...,Chi-Yi Chen,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2013-06-01,2013,<NA>,parent,openalex::W999790414,chen_chi_yi,investigation of the formalism of particle dyn...,chen|c,None


In [24]:
print(data_group_merge["records_hierarchy_copy"].value_counts(dropna=False).head(60))

records_hierarchy_copy
parent                                          7931373
review                                           175443
part_of                                          149454
version                                          105977
publish_version                                    9351
mirror (arXiv)                                     8286
mirror (AgEcon Search)                             6702
mirror (Open Science Framework)                    4356
mirror (PsyArXiv)                                  4304
mirror (INA-Rxiv)                                  3768
mirror (SocArXiv)                                  3395
child                                              2028
mirror (ResearchGate)                              1633
mirror (Law Archive)                                935
mirror (EarthArXiv)                                 410
correction                                          355
mirror (Thesis Commons)                             346
comment                  

In [25]:
print(data_group_merge["parent_record_id"].value_counts(dropna=False).head(60))

parent_record_id
<NA>                                         7442983
None                                          326687
datacite::10.5281/zenodo.14289192                701
datacite::10.5281/zenodo.15609432                569
datacite::10.22004/ag.econ.138684                449
datacite::10.22004/ag.econ.136112                389
datacite::10.22004/ag.econ.286599                300
datacite::10.5281/zenodo.15832876                285
datacite::10.5281/zenodo.15161515                260
crossref::10.31219/osf.io/ed2a9                  209
crossref::10.32388/174914                        183
datacite::10.5281/zenodo.6784019                 181
datacite::10.5281/zenodo.14599183                175
datacite::10.5281/zenodo.15631517                166
crossref::10.2139/ssrn.2721313                   163
datacite::10.5281/zenodo.15690627                158
crossref::10.21055/preprints-3112015             144
datacite::10.5281/zenodo.1297654                 140
datacite::10.5281/zenodo.1717

In [26]:
pattern = "fuzzy::las"


mask = data_group_merge['dup_group_id'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

0


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last


In [28]:
# result['relations_json'][281751]

In [29]:
data_group_merge[data_group_merge["record_id"]=='crossref::10.26434/chemrxiv-2021-cj17s']

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
281806,crossref::10.26434/chemrxiv-2021-cj17s,ChemRxiv,crossref,10.26434/chemrxiv-2021-cj17s,https://doi.org/10.26434/chemrxiv-2021-cj17s,https://chemrxiv.org/engage/chemrxiv/article-d...,First-Principles Plane-Wave-Based Exploration ...,"Ertural, Christina; Stoffel, Ralf; Müller, Pet...",RWTH Aachen University,None,None,None,,,,,false,None,None,None,None,parent,2021-07-26,2021,<NA>,parent,crossref::10.26434/chemrxiv-2021-6cxlq,None,first principles plane wave based exploration ...,dronskowski|r;ertural|c;muller|p;stoffel|r;vogt|c,None


In [30]:
data_group_merge[data_group_merge["parent_record_id"]=='crossref::10.26434/chemrxiv-2022-fd190']

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
282821,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None
282876,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282883,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
283800,crossref::10.26434/chemrxiv-2022-fd190-v5,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v5,https://doi.org/10.26434/chemrxiv-2022-fd190-v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-08-24,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
309786,crossref::10.26434/chemrxiv-2022-fd190-v6,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v6,https://doi.org/10.26434/chemrxiv-2022-fd190-v6,https://chemrxiv.org/engage/chemrxiv/article-d...,Can the CO2 Reduction Reaction be Improved on ...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-preprint-of"": [{""asserted-by"": ""subject"",...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,10.1021/acscatal.2c04200,,,true,None,None,None,None,parent - duplicate (ChemRxiv),2022-10-18,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,c

In [31]:
data_group_merge[data_group_merge["dup_group_id"]=='crossref::10.26434/chemrxiv-2022-fd190']

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
282550,crossref::10.26434/chemrxiv-2022-fd190,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190,https://doi.org/10.26434/chemrxiv-2022-fd190,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,None,None,,,,,false,None,None,None,None,parent,2022-03-18,2022,<NA>,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282821,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None
282876,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282883,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
283800,crossref::10.26434/chemrxiv-2022-fd190-v5,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v5,https://doi.org/10.26434/chemrxiv-2022-fd190-v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-08-24,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
309786,crossref::10.26434/chemrxiv-2022-fd1

In [33]:
data_group_merge[data_group_merge["dup_group_id"]=='crossref::10.26434/chemrxiv-2022-fd190']['title'].value_counts()

title
Selectivity and Intrinsic Activity of Functionalized Cu Surfaces: Can the CO2 Reduction Reaction be Improved on Cu?    5
Can the CO2 Reduction Reaction be Improved on Cu: Selectivity and Intrinsic Activity of Functionalized Cu Surfaces     1
Name: count, dtype: int64

In [34]:
pattern = "chemrxiv.11846943"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

9


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
276994,crossref::10.26434/chemrxiv.11846943.v1,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v1,https://doi.org/10.26434/chemrxiv.11846943.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Analysis of Whole Genome Sequences and Homolog...,"Shanker, Arun; Bhanu, Divya; Alluri, Anajani",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (Open Science Framework),2020-02-13,2020,crossref::10.31219/osf.io/2zuea,parent,crossref::10.26434/chemrxiv.11846943.v1,alluri_anajani;arun_shanker;bhanu_divya,analysis of whole genome sequences and homolog...,None,None
277030,crossref::10.26434/chemrxiv.11846943.v2,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v2,https://doi.org/10.26434/chemrxiv.11846943.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Analysis of Whole Genome Sequences and Homolog...,"Shanker, Arun; Bhanu, Divya; Alluri, Anajani",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (Open Science Framework),2020-02-14,2020,crossref::10.31219/osf.io/2zuea,parent,crossref::10.26434/chemrxiv.11846943.v1,alluri_anajani;arun_shanker;bhanu_divya,analysis of whole genome sequences and homolog...,None,None
277064,crossref::10.26434/chemrxiv.11846943.v3,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v3,https://doi.org/10.26434/chemrxiv.11846943.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Analysis of Whole Genome Sequences and Homolog...,"Shanker, Arun; Alluri, Anjani; Bhanu, Divya",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-02-17,2020,crossref::10.26434/chemrxiv.11846943.v2,parent,crossref::10.26434/chemrxiv.11846943.v1,alluri_anjani;arun_shanker;bhanu_divya,analysis of whole genome sequences and homolog...,None,None
277093,crossref::10.26434/chemrxiv.11846943.v4,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v4,https://doi.org/10.26434/chemrxiv.11846943.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Whole Genome Sequences Analysis and Homology M...,"Shanker, Arun; Alluri, Anjani; Bhanu, Divya",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (Open Science Framework),2020-02-21,2020,crossref::10.31219/osf.io/2zuea,parent,crossref::10.26434/chemrxiv.11846943.v1,alluri_anjani;arun_shanker;bhanu_divya,whole genome sequences analysis and homology m...,alluri|a;bhanu|d;shanker|a,None
277128,crossref::10.26434/chemrxiv.11846943.v5,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v5,https://doi.org/10.26434/chemrxiv.11846943.v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Whole Genome Sequence Analysis and Homology Mo...,"Shanker, Arun; Alluri, Anjani; Bhanu, Divya",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-03-10,2020,crossref::10.26434/chemrxiv.11846943.v4,parent,crossref::10.26434/chemr

In [35]:
result['records_hierarchy'].value_counts()

records_hierarchy
parent - duplicate (ChemRxiv)                  6
parent - duplicate (Open Science Framework)    3
Name: count, dtype: int64

In [36]:
result['title'].value_counts()

title
Whole Genome Sequence Analysis and Homology Modelling of a 3C Like Peptidase and a Non-Structural Protein 3 of the SARS-CoV-2 Shows Protein Ligand Interaction with an Aza-Peptide and a Noncovalent Lead Inhibitor with Possible Antiviral Properties                       5
Analysis of Whole Genome Sequences and Homology Modelling of a 3-C Like Peptidase and a Non-Structural Protein of the Novel Coronavirus COVID-19 Shows Protein Ligand Interaction with an Aza-Peptide and a Noncovalent Lead Inhibitor with Possible Antiviral Properties    2
Analysis of Whole Genome Sequences and Homology Modelling of a 3C Like Peptidase and a Non-Structural Protein of the Novel Coronavirus COVID-19 Shows Protein Ligand Interaction with an Aza-Peptide and a Noncovalent Lead Inhibitor with Possible Antiviral Properties     1
Whole Genome Sequences Analysis and Homology Modelling of a 3C Like Peptidase and a Non-Structural Protein 3 of the SARS-CoV-2 Shows Protein Ligand Interaction with an Aza-Peptide a

In [37]:
result['authors_flat'].value_counts()

authors_flat
Shanker, Arun; Alluri, Anjani; Bhanu, Divya                      3
Shanker, Arun; Bhanu, Divya; Alluri, Anjani                      3
Shanker, Arun; Bhanu, Divya; Alluri, Anajani                     2
Shanker, Arun; Bhanu, Divya; Alluri, Anjani; Gupta, Samriddhi    1
Name: count, dtype: int64

In [38]:
pattern = "10.26434/chemrxiv-2022-fd190"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

6


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
282550,crossref::10.26434/chemrxiv-2022-fd190,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190,https://doi.org/10.26434/chemrxiv-2022-fd190,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,None,None,,,,,false,None,None,None,None,parent,2022-03-18,2022,<NA>,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282821,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None
282876,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282883,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
283800,crossref::10.26434/chemrxiv-2022-fd190-v5,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v5,https://doi.org/10.26434/chemrxiv-2022-fd190-v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-08-24,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
309786,crossref::10.26434/chemrxiv-2022-fd1

In [39]:
pattern = "10.26434/chemrxiv-2022-fd190"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

6


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
282550,crossref::10.26434/chemrxiv-2022-fd190,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190,https://doi.org/10.26434/chemrxiv-2022-fd190,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,None,None,,,,,false,None,None,None,None,parent,2022-03-18,2022,<NA>,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282821,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None
282876,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
282883,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
283800,crossref::10.26434/chemrxiv-2022-fd190-v5,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v5,https://doi.org/10.26434/chemrxiv-2022-fd190-v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-08-24,2022,crossref::10.26434/chemrxiv-2022-fd190,parent,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None
309786,crossref::10.26434/chemrxiv-2022-fd1

In [40]:
result['title'].value_counts()

title
Selectivity and Intrinsic Activity of Functionalized Cu Surfaces: Can the CO2 Reduction Reaction be Improved on Cu?    5
Can the CO2 Reduction Reaction be Improved on Cu: Selectivity and Intrinsic Activity of Functionalized Cu Surfaces     1
Name: count, dtype: int64

In [41]:
result['authors_flat'].value_counts()

authors_flat
Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong; Bagger, Alexander; Vang Lauritsen, Jeppe; Uttrup Pedersen, Steen; Daasbjerg, Kim; Rossmeisl, Jan                             5
Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong; Bagger, Alexander; Rosas-Hernández, Alonso; Vang Lauritsen, Jeppe; Uttrup Pedersen, Steen; Daasbjerg, Kim; Rossmeisl, Jan    1
Name: count, dtype: int64

In [42]:
sample_titles = data_group_merge.sample(5)[['title', 'authors_flat']].title
data_group_merge[data_group_merge.title.isin(sample_titles)][['title','authors_flat','records_hierarchy','date_first_seen']]

,title,authors_flat,records_hierarchy,date_first_seen
424970,HydroShare tools and recommended practices for...,"Tarboton, David; Idaszak, Ray; Horsburgh, Jeff...",parent,2018-12-18
3345105,New Paradigm in Mortgage Loan Advice,"Otterstedt, Margrét Sesselja; Rasmussen, Kouro...",parent,2012-01-01
3944498,Effect of Austenitizing Temperature and Coolin...,"Cao, Kuo; Wang, Zhigang; Liu, Pengfei; Zhao, A...",parent,2023-01-01
4034912,Development of Composites Based on Linear Low-...,"León, Lumirca Del Valle Espinoza; Escócio, Vi...",parent,2023-01-01
5803901,An ab initio design of cluster-assembled silic...,"Guo, Lingju; Zheng, Xiaohong; Liu, Chunsheng; ...",parent,2022-03-14


In [43]:
pattern = "10.26434/chemrxiv.13102877"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

32


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
279942,crossref::10.26434/chemrxiv.13102877.v1,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v1,https://doi.org/10.26434/chemrxiv.13102877.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-10-23,2020,<NA>,parent,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,datta|s,None
279972,crossref::10.26434/chemrxiv.13102877.v2,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v2,https://doi.org/10.26434/chemrxiv.13102877.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-10-27,2020,crossref::10.26434/chemrxiv.13102877.v1,parent,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,None,None
279999,crossref::10.26434/chemrxiv.13102877.v3,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v3,https://doi.org/10.26434/chemrxiv.13102877.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-11-06,2020,<NA>,parent,crossref::10.26434/chemrxiv.13102877.v10,datta_shoumen,aptamers for detection and diagnostics add is ...,datta|s,None
280030,crossref::10.26434/chemrxiv.13102877.v4,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v4,https://doi.org/10.26434/chemrxiv.13102877.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-09,2020,crossref::10.26434/chemrxiv.13102877.v3,parent,crossref::10.26434/chemrxiv.13102877.v10,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None
280070,crossref::10.26434/chemrxiv.13102877.v5,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v5,https://doi.org/10.26434/chemrxiv.13102877.v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-11,2020,crossref::10.26434/chemrxiv.13102877.v3,parent,crossref::10.26434/chemrxiv.13102877.v10,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None
280100,crossref::10.26434/chemrxiv.13102877.v6,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v6,https://doi.org/10.26434/chemrxiv.13102877.v6,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-16,2020,crossref::10.26434/chemrxiv.13102877.v3,parent,crossref::10.26434/chemrxiv.131028

In [44]:
result['title'].value_counts()

title
Aptamers for Detection and Diagnostics (ADD) is a proposed mobile app acquiring optical data from conjugated quantum nanodots to identify molecules indicating presence of SARS-CoV-2 virus: Why public health and healthcare need smartphone sensors as a platform for early detection and prevention    28
Aptamers for Detection and Diagnostics (ADD): Can mobile systems process optical data from aptamer sensors to identify molecules indicating presence of SARS-CoV-2 virus? Should healthcare explore aptamers as drugs for prevention as well as its use as adjuvants with antibodies and vaccines?         2
Aptamers for Detection and Diagnostics (ADD): Proposed Mobile App Acquiring Optical Data from Aptamers Conjugated with Quantum Nanodots May Detect Harmful Molecules as well as SARS-CoV-2                                                                                                                 1
Aptamers for Detection and Diagnostics (ADD): Proposed Mobile App Acquiring Optical Data fr

In [45]:
result['authors_flat'].value_counts()

authors_flat
Datta, Shoumen    32
Name: count, dtype: int64

In [46]:
result['dup_group_id'].value_counts()

dup_group_id
crossref::10.26434/chemrxiv.13102877.v10      28
crossref::10.26434/chemrxiv.13102877.v1        2
crossref::10.26434/chemrxiv-2021-l393t-v33     2
Name: count, dtype: int64

In [47]:
result['parent_record_id'].value_counts()

parent_record_id
crossref::10.26434/chemrxiv.13102877.v3     27
crossref::10.26434/chemrxiv.13102877.v1      1
crossref::10.26434/chemrxiv.13102877.v31     1
Name: count, dtype: int64